In [1]:
import pandas as pd

# Cuota parte

In [2]:
def cuota_parte(cuota,capacidad,SA,PE,SIN,Rea):
    """Función que calcula el contrato de reaguro cuota parte
    
    Inputs:
    Cuota: float
    
    Es el porcentaje de la cuota parte
    
    Capacidad: float
    
    Es la capacidad del contrato
    
    SA: float
    
    Es la suma asegurada
    
    PE: float
    
    Es el valor de la prima emitida
    
    SIN: float
    
     Es el valor del siniestro
     
     Rea:list
     
     Lista con los porcentajes de las reaseguradoras
    
    Output:
    DataFrame
    
    """
    minimo=min(cuota*SA,cuota*capacidad)
    if minimo==cuota*SA:
        Sa=[minimo]
        for ase in Rea:
            Sa+=[(SA-Sa[0])*ase]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]

    else:
        facultativo=SA-capacidad
        Sa=[capacidad*cuota]
        for ase in Rea:
                Sa+=[capacidad*(1-cuota)*ase]
        Sa+=[facultativo]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]+['Facultativo']

    df= pd.DataFrame([Sa,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    return df

In [3]:
#Ejemplo 1
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,108000.0,4500.0,21600.0
CP Reaseguradora 2,252000.0,10500.0,50400.0


In [4]:
#Ejempolo 2
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[1,0])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,360000.0,15000.0,72000.0
CP Reaseguradora 2,0.0,0.0,0.0


In [5]:
#Ejempolo 3
cuota_parte(0.6,2_000_000,3_500_000,25_000,250_000,[0.4,0.6])

,Suma Asegurada,Prima,Siniestro
Retención,1200000.0,8571.428571,85714.285714
CP Reaseguradora 1,320000.0,2285.714286,22857.142857
CP Reaseguradora 2,480000.0,3428.571429,34285.714286
Facultativo,1500000.0,10714.285714,107142.857143


In [6]:
#Ejemplo 4
cuota_parte(0.4,2_000_000,2_500_000,70_000,500_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,800000.0,22400.0,160000.0
CP Reaseguradora 1,360000.0,10080.0,72000.0
CP Reaseguradora 2,840000.0,23520.0,168000.0
Facultativo,500000.0,14000.0,100000.0


In [7]:
#Ejemplo 5
cuota_parte(0.3,3_000_000,4_500_000,100_000,800_000,[0.4,0.5,0.1])

,Suma Asegurada,Prima,Siniestro
Retención,900000.0,20000.000000,160000.000000
CP Reaseguradora 1,840000.0,18666.666667,149333.333333
CP Reaseguradora 2,1050000.0,23333.333333,186666.666667
CP Reaseguradora 3,210000.0,4666.666667,37333.333333
Facultativo,1500000.0,33333.333333,266666.666667


# Excedentes

In [8]:
def excedentes(limite, lineas,SA,PE,Sin,REA):
    """
    Función que calcula los contratos de excedentes 
    
    inputs:
    
    limite: float
    
    Es el limite de rentención por líneas
    
    lineas: float
    
    Es el limite de lineas que tiene el contrato
    
    SA: float
    
    Es el valor de la suma asegurada
    
    PE:float
    
    Es el valor de la prima emitida
    
    Sin: foat
    
    Es el valor del siniestro
    
    REA:list
    
    Lista con los porcentajes de las reaseguradoras
    
    Output:
    DataFrame
    """
    capacidad=limite*lineas
    if sum(REA)>1:
        REA=[r/100 for r in REA]
    if capacidad>SA:
        exceden=[limite]+[(SA-limite)*re for re in REA]
        columnas=["Retención"]+['Excedente Reaseguradora '+str(i+1) for i in range(len(REA))]
    else:
        exceden=[limite]+[(capacidad)*re for re in REA]
        facultativo=SA-sum(exceden)
        exceden=exceden+[facultativo]
        columnas=["Retención"]+['Excedente Reaseguradora '+str(i+1) for i in range(len(REA))]+['Facultativo']
    porcentajes=[exc/SA for exc in exceden]
    primas=[PE*por for por in porcentajes]
    siniestros=[Sin*por for por in porcentajes]

    df= pd.DataFrame([exceden,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    
    return df

In [9]:
#Ejemplo 1
excedentes(500_000,10,600_000,25_000,300_000,[1])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,20833.333333,250000.0
Excedente Reaseguradora 1,100000.0,4166.666667,50000.0


In [10]:
#Ejemplo 2
excedentes(500_000,10, 3_000_000,80_000, 300_000,[35,65])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,13333.333333,50000.0
Excedente Reaseguradora 1,875000.0,23333.333333,87500.0
Excedente Reaseguradora 2,1625000.0,43333.333333,162500.0


In [11]:
#Ejemplo 3
excedentes(500_000,10,7_000_000,160_000,300_000 ,[20,30,50])

,Suma Asegurada,Prima,Siniestro
Retención,500000.0,11428.571429,21428.571429
Excedente Reaseguradora 1,1000000.0,22857.142857,42857.142857
Excedente Reaseguradora 2,1500000.0,34285.714286,64285.714286
Excedente Reaseguradora 3,2500000.0,57142.857143,107142.857143
Facultativo,1500000.0,34285.714286,64285.714286


In [12]:
#Ejemplo 4
excedentes(600_000,11,10_000_000,280_000,900_000,[25,35,30,10])

,Suma Asegurada,Prima,Siniestro
Retención,600000.0,16800.0,54000.0
Excedente Reaseguradora 1,1650000.0,46200.0,148500.0
Excedente Reaseguradora 2,2310000.0,64680.0,207900.0
Excedente Reaseguradora 3,1980000.0,55440.0,178200.0
Excedente Reaseguradora 4,660000.0,18480.0,59400.0
Facultativo,2800000.0,78400.0,252000.0


#  Cuota Parte y Excedente 


In [13]:
def cuota_excedente(cuota_par,reten,lineas,SA,PE,SIN,REAS,EXCEDEN):
    """
    Función que calcula el reaseguro de cuota parte junto con el de excedente 
    
    Inputs:
    
    cuota_par:float
    
    Es el porcentaje con el que se se queda la aseguradora en el contrato cuota parte
    
    reten:float
    
    Es el límite de retención del contrato cuota parte
    
    lineas:int
     
     Es el número de lineas par el contrato de excedentes
     
     SA:int
     
     Valor de la suma asegurada
     
     PE:int
     
     Valor de la prima emitida
     
     SIN:int
     
     Valor del siniestro
     
     REAS:list
     
     Lista con el valor del porcentaje de  participación de las reaseguradoras para los contratos cuota parte
     
     EXCEDEN:list
     
     Lista con el valor del porcentaje de  participación de las reaseguradoras para los contratos de excedente

     
     Output:
     
     Dataframe
    """
    reten_cuota=min(cuota_par*reten,cuota_par*SA)
    columnas=['Retención']+['CP Reasegurador '+str(i+1) for i in range(len(REAS))]
    if reten_cuota==cuota_par*SA:
        cuota_partes=[SA*(1-cuota_par)*r for r in REAS]
        general=[reten_cuota]+cuota_partes
    else:
        capacidad=reten*lineas
        cuotas_partes=[reten*(1-cuota_par)*r for r in REAS]
        columnas+=['Exc Reasegurador '+str(j+1) for j in range(len(REAS))]
        if SA<=(reten+capacidad):
            excedentes=[(SA-reten)*exc for exc in EXCEDEN]
            general=[reten_cuota]+cuotas_partes+excedentes
        else:
            excedentes=[capacidad*exc for exc in EXCEDEN]
            general=[reten_cuota]+cuotas_partes+excedentes
        if SA>sum(general):
                facultativo=SA-sum(general)
                general+=[facultativo]
                columnas+=['Facultativo']        
    porcentajes=[g/SA for g in general]
    primas=[PE*p for p in porcentajes]
    siniestros=[SIN*p for p in porcentajes]
    df=pd.DataFrame([general,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    return df        

In [14]:
#Ejemplo 1
cuota_excedente(0.3,1_000_000,10,700_000,10_000,500_000,[0.3,0.7],[0.6,0.4])

,Suma Asegurada,Prima,Siniestro
Retención,210000.0,3000.0,150000.0
CP Reasegurador 1,147000.0,2100.0,105000.0
CP Reasegurador 2,343000.0,4900.0,245000.0


In [15]:
#Ejemplo 2
cuota_excedente(0.3,1_000_000,10,5_000_000,60_000,600_000,[0.3,0.7],[0.6,0.4])

,Suma Asegurada,Prima,Siniestro
Retención,300000.0,3600.0,36000.0
CP Reasegurador 1,210000.0,2520.0,25200.0
CP Reasegurador 2,490000.0,5880.0,58800.0
Exc Reasegurador 1,2400000.0,28800.0,288000.0
Exc Reasegurador 2,1600000.0,19200.0,192000.0


In [16]:
#Ejemplo 3
cuota_excedente(0.3,1_000_000,10,7_000_000,125_000,1_000_000,[0.3,0.7],[0.6,0.4])

,Suma Asegurada,Prima,Siniestro
Retención,300000.0,5357.142857,42857.142857
CP Reasegurador 1,210000.0,3750.000000,30000.000000
CP Reasegurador 2,490000.0,8750.000000,70000.000000
Exc Reasegurador 1,3600000.0,64285.714286,514285.714286
Exc Reasegurador 2,2400000.0,42857.142857,342857.142857


In [17]:
#Ejemplo 4
cuota_excedente(0.3,1_000_000,10,12_000_000,160_000,2_500_000,[0.3,0.7],[0.6,0.4])


,Suma Asegurada,Prima,Siniestro
Retención,300000.0,4000.000000,6.250000e+04
CP Reasegurador 1,210000.0,2800.000000,4.375000e+04
CP Reasegurador 2,490000.0,6533.333333,1.020833e+05
Exc Reasegurador 1,6000000.0,80000.000000,1.250000e+06
Exc Reasegurador 2,4000000.0,53333.333333,8.333333e+05
Facultativo,1000000.0,13333.333333,2.083333e+05


In [18]:
#Ejemplo 5
cuota_excedente(0.4,1_000_000,10,14_000_000,200_000,3_800_000,[0.6,0.4],[0.2,0.8])

,Suma Asegurada,Prima,Siniestro
Retención,400000.0,5714.285714,1.085714e+05
CP Reasegurador 1,360000.0,5142.857143,9.771429e+04
CP Reasegurador 2,240000.0,3428.571429,6.514286e+04
Exc Reasegurador 1,2000000.0,28571.428571,5.428571e+05
Exc Reasegurador 2,8000000.0,114285.714286,2.171429e+06
Facultativo,3000000.0,42857.142857,8.142857e+05


# Excedentes Cuota Parte y Taba de Plenos

In [19]:
def excedentes_cuota_plenos(cuota_par,lim_rente,lineas,Plenos,SA,PE):
    """
    Función que cálcula el reaseguro de un contrato de cuota parte, excedente y tabla de plenos
    
    Inputs:
    
    cuota_par:float
    Es el porcentaje de retención de la asegudora en el contrato de cuota parte
    
    lim_rente:int
    Es el límite de retención en el cotrato de cuota parte
    
    lineas:int
    Es el número de líneas que teiene el contrato de excedentes
    
    plenos:list
    Lista con los porcentajes de la tabla de plenos
    
    SA: list
    Lista con  las sumas asegruradas, con los riesgos en las tablas de pleno
    
    PE:list
    Lista con el valor de las primas emitidas
    
    Output:
    DataFrame
    
    """
    def porcenta_tuple(terna): return [i/sum(terna) for i in terna]
    def mult_list(lista,n): return [round(n*l*1_000,2) for l in lista]
    reten_plenos=[p*lim_rente for p in Plenos]
    reten_sa=[rt*cuota_par for rt in reten_plenos]
    cuota_partes=[rt*(1-cuota_par) for rt in reten_plenos]
    excedentes=[sa-rt for sa,rt in zip(SA,reten_plenos)]
    general=[(r,c,e) for r,c,e in zip(reten_sa,cuota_partes,excedentes)]
    porcentajes=[porcenta_tuple(tupla) for tupla in general]
    primas=[mult_list(por,Prim) for por,Prim in zip(porcentajes,PE)]    
    df=pd.DataFrame([reten_sa,cuota_partes,excedentes],
                        columns=["Riesgo "+str(i+1) for i in range(len(Plenos))]).T
    df.columns=["Retención",'Cuota Parte','Excedentes']
    df=df.T
    df_primas=pd.DataFrame(primas,columns=["Retención",'Cuota Parte','Excedentes']).T
    df_primas.columns=["Riesgo "+str(i+1) for i in range(len(Plenos))]    
    return df,df_primas

In [20]:
excedentes_cuota_plenos(0.5,1_000_000,10,[0.25,0.5,0.8],[600_000,3_000_000,7_000_000],[5,25_000,60_000])

(             Riesgo 1   Riesgo 2   Riesgo 3
 Retención    125000.0   250000.0   400000.0
 Cuota Parte  125000.0   250000.0   400000.0
 Excedentes   350000.0  2500000.0  6200000.0,
              Riesgo 1     Riesgo 2     Riesgo 3
 Retención     1041.67   2083333.33   3428571.43
 Cuota Parte   1041.67   2083333.33   3428571.43
 Excedentes    2916.67  20833333.33  53142857.14)

# WXL

In [21]:
def wxl(prioridad,WXL,siniestro):
    '''
    Función que cálcula el contrato de reaguro WXL
    
    Inputs:
    
    Priorida:int
    
    Es la prioridad del contrato
    
    WXL:list
    
    Lista con las capas del contrato
    
    siniestro:int
    
    Monto del siniestro
    
    Output:
    list
    '''
    nueva_prioridad=min(siniestro,prioridad)
    if nueva_prioridad == siniestro:
        return "Prioridad = " +str(siniestro)
    else:
        acum=[nueva_prioridad]
        for capa in WXL:
            if sum(acum)<siniestro:
                acum+=[min(capa,siniestro-sum(acum))]
        return acum        

In [22]:
#Ejemplo 1
wxl(500_000,[500_000],300_000)

'Prioridad = 300000'

In [23]:
#Ejemplo 2
wxl(500_000,[500_000],600_000)

[500000, 100000]

In [24]:
#Ejemplo 3
wxl(500_000,[200_000,500_000],600_000)

[500000, 100000]

In [25]:
#Ejemplo 4
wxl(500_000,[200_000,500_000],700_000)

[500000, 200000]

In [26]:
#Ejemplo 5
wxl(500_000,[200_000,500_000],850_000)

[500000, 200000, 150000]

In [27]:
#Ejemplo 6
wxl(400_000,[100_000,150_000,200_000],700_000)

[400000, 100000, 150000, 50000]

# WXL & Excedentes 

In [28]:
def wxl_excedente(lim_reten,lineas,prioridad,WXL,REA,SA,PE,SIN):
    '''
    Función que cálcula el contrato de reaseguro wxl y excedente
    
    Inputs:
    
    lim_reten:int
    
    Limite de retención
    
    lineas:int
    
    Número de lineas del contrato
    
    prioridad:int
    
    Prioridad en el contrato WXL
    
    WXL:list
    
    Lista conlas capas del contrato WXL
    
    REA:list
    
    Lista con los porcentajas de las reaseguradoras
    
    SA,PE,SIN
    
    Output
    '''
    df_excedente=excedentes(lim_reten,lineas, SA, PE, SIN, REA)
    siniestro_retencion=df_excedente['Siniestro']['Retención']
    return df_excedente,wxl(prioridad,WXL,siniestro_retencion)
    
    

In [29]:
wxl_excedente(5_000_000,10, 1_000_000,[3_000_000,1_000_000],
              [0.4,0.25,0.35],20_000_000,250_000,12_000_000)

(                           Suma Asegurada    Prima  Siniestro
 Retención                       5000000.0  62500.0  3000000.0
 Excedente Reaseguradora 1       6000000.0  75000.0  3600000.0
 Excedente Reaseguradora 2       3750000.0  46875.0  2250000.0
 Excedente Reaseguradora 3       5250000.0  65625.0  3150000.0,
 [1000000, 2000000.0])

In [30]:
wxl_excedente(5_000_000,10, 1_000_000,[1_500_000,1_000_000], 
              [0.4,0.25,0.35],20_000_000,250_000,12_000_000)

(                           Suma Asegurada    Prima  Siniestro
 Retención                       5000000.0  62500.0  3000000.0
 Excedente Reaseguradora 1       6000000.0  75000.0  3600000.0
 Excedente Reaseguradora 2       3750000.0  46875.0  2250000.0
 Excedente Reaseguradora 3       5250000.0  65625.0  3150000.0,
 [1000000, 1500000, 500000.0])